# import lib

In [1]:
#Data management
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
import re
import string
import os
from tqdm import tqdm
import glob


#data visualize
import matplotlib.pyplot as plt
import seaborn as sns

pythai lib

In [4]:
!pip install pythainlp -qq
!pip install epitran -qq
!pip install sklearn_crfsuite -qq
!pip install tensorflow deepcut -qq
!pip install attacut -qq
!pip install pysqlite3

     |████████████████████████████████| 40 kB 3.6 MB/s 
  Created wheel for pysqlite3: filename=pysqlite3-0.4.6-cp37-cp37m-linux_x86_64.whl size=132573 sha256=48da8dda575187fee15b31819eed6b94f84ca4f6eb20a56069249507fa582878
  Stored in directory: /root/.cache/pip/wheels/75/38/92/2f94f7efeced4164de2484de2cad2380044c7e6efe95b311f5
Successfully built pysqlite3


# Test Tokenize

In [5]:
from pythainlp import sent_tokenize, word_tokenize
text = "แปลกแต่จริงสิ่งนี้ไม่น่าจะเกิดขึ้นกับฉันบอกกับฉันว่าเธอทำได้ยังไงเปลี่ยนคนที่คอยปิดหัวใจให้รักเธอง่ายดายศีรษะ"
print("sent_tokenize:", sent_tokenize(text))
print("newmm    :", word_tokenize(text, engine="newmm"))
print("longest  :", word_tokenize(text, engine="longest"))
print("multi_cut:", word_tokenize(text, engine="multi_cut"))
print("deepcut  :", word_tokenize(text, engine="deepcut"))
print("atta     :", word_tokenize(text, engine="attacut"))
word_newmm =  word_tokenize(text, engine="newmm")
word_longest =  word_tokenize(text, engine="longest")
word_multi = word_tokenize(text, engine="multi_cut")
word_deepcut =  word_tokenize(text, engine="deepcut")
word_atta = word_tokenize(text, engine="attacut")

sent_tokenize: ['แปลกแต่จริงสิ่งนี้ไม่น่าจะเกิดขึ้นกับฉันบอกกับฉันว่าเธอทำได้ยังไงเปลี่ยนคนที่คอยปิดหัวใจให้รักเธอง่ายดายศีรษะ']
newmm    : ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่าจะ', 'เกิด', 'ขึ้นกับ', 'ฉัน', 'บอก', 'กับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำได้', 'ยังไง', 'เปลี่ยนคน', 'ที่', 'คอย', 'ปิด', 'หัวใจ', 'ให้', 'รัก', 'เธอ', 'ง่ายดาย', 'ศีรษะ']
longest  : ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่าจะ', 'เกิดขึ้น', 'กับ', 'ฉัน', 'บอก', 'กับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำได้', 'ยังไง', 'เปลี่ยนคน', 'ที่', 'คอย', 'ปิด', 'หัวใจ', 'ให้', 'รัก', 'เธอ', 'ง่ายดาย', 'ศีรษะ']
multi_cut: ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่าจะ', 'เกิดขึ้นกับ', 'ฉัน', 'บอกกับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำได้', 'ยังไง', 'เปลี่ยนคนที่', 'คอย', 'ปิด', 'หัวใจ', 'ให้', 'รัก', 'เธอ', 'ง่ายดายศีรษะ']
deepcut  : ['แปลก', 'แต่', 'จริง', 'สิ่ง', 'นี้', 'ไม่', 'น่า', 'จะ', 'เกิด', 'ขึ้น', 'กับ', 'ฉัน', 'บอก', 'กับ', 'ฉัน', 'ว่า', 'เธอ', 'ทำ', 'ได้', 'ยัง', 'ไง', 'เปลี่ยน', 'คน', 'ที่', 'คอย', 'ปิด', '

# Corpus

pythai lib

In [6]:
from pythainlp import sent_tokenize, word_tokenize
from pythainlp import thai_digits, thai_letters
from pythainlp.spell import NorvigSpellChecker
from pythainlp.corpus import download , get_corpus_path , get_corpus

In [7]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
len(checker.dictionary())
checker.dictionary()

dict_items([('ยโศธ', 22), ('มุ', 375), ('วสี', 29), ('ผกผัน', 110), ('วหา', 4), ('ระฆัง', 557), ('รัตนะ', 47), ('โกหก', 1167), ('พฤติ', 6), ('จูน', 120), ('จุ๊ย', 20), ('เบล้าส์', 3), ('ลูกหลวง', 20), ('เบลต์', 3), ('โครงการหลวง', 50), ('เจ้าหลวง', 203), ('คั้น', 321), ('แคร์', 498), ('แซนซ์', 3), ('เมาน์', 3), ('แมกไม้', 62), ('คิ่ว', 4), ('ไต่ไม้', 4), ('กันเกรา', 5), ('แอว', 3), ('ววววว', 8), ('ดรัล', 6), ('แอนติบอดี', 13), ('ริมทาง', 185), ('ติดตลาด', 49), ('แฟรงส์', 8), ('ฮิก', 5), ('แถว', 4909), ('ฟม', 7), ('ภาระติดพันในอสังหาริมทรัพย์', 4), ('บุรพาจารย์', 2), ('ชิม', 1024), ('ผู้แทนโดยชอบธรรม', 114), ('พรั่งพรู', 95), ('ขุ่นข้อง', 32), ('ล่วง', 449), ('เรด', 118), ('ห่', 3), ('ศดาร', 2), ('เอว', 1070), ('หมื่อ', 6), ('กิน', 26786), ('วาตภัย', 22), ('ทศพล', 20), ('พรุ', 42), ('บรรจุ', 3217), ('วิรัช', 117), ('วด็อก', 2), ('ติณห์', 79), ('บัวตูม', 24), ('ตะเกียงรั้ว', 16), ('พิธีกรรม', 1150), ('คอนโดมีเนียม', 5), ('เก้', 327), ('เมธาวี', 346), ('เป็นเนื้อเป็นตัว', 4), ('จ็อก', 2),

# get Corpus Class

In [8]:
class Corpus():
  
  def __init__(self,dict_word_fre):
    self.dictt = dict_word_fre
    self.word = list(dict_word_fre.keys())
    self.fre = list(dict_word_fre.values())
    print("Complete init corpus")

  def __len__(self):
    return len(self.word)

  def __getitem__(self,word):
    return self.dictt[word]

class Corpus_from_dow(Corpus):
  def __init__(self,name):
    dict_cor = self.dow(name)
    Corpus.__init__(self,dict_cor)
  def dow(self,name):
    download(name)
    path = get_corpus_path(name)
    cor = get_corpus(path)
    word = ["".join(x.split("\t")[:-1]) for x in cor]
    fre = [int(x.split("\t")[-1]) for x in cor]
    dict_cor = dict(zip(word, fre))
    return dict_cor

In [9]:
checker = NorvigSpellChecker()  # use default filter (remove any word with number or non-Thai character)
unigrame = checker.dictionary()

In [10]:
unigrame = Corpus(dict(checker.dictionary()))

Complete init corpus


In [11]:
trigrame = Corpus_from_dow("tnc_trigram_word_freqs")
bigrame = Corpus_from_dow("tnc_bigram_word_freqs")

Corpus: tnc_trigram_word_freqs
- Downloading: tnc_trigram_word_freqs 2017


100%|██████████| 151914848/151914848 [00:02<00:00, 75484080.97it/s]


Complete init corpus
Corpus: tnc_bigram_word_freqs
- Downloading: tnc_bigram_word_freqs 2017


100%|██████████| 42200711/42200711 [00:00<00:00, 127207126.76it/s]


Complete init corpus


In [12]:
a = trigrame.dictt.get('f3fewfwf4')
print(a)

None


In [13]:
trigrame.dictt.get('สวัสดีครับท่าน')

10

In [ ]:
from pythainlp.tokenize import word_tokenize

N = sum(trigrame.dictt.values())

def is_overlapped(a,b):
  if a[0] > b[0]: a,b = b,a
  if a[1] > b[0]: return True
  return False

def get_best_prop_sentence(sentence, candidates):
  sentence = list(sentence)
  window_size = 3
  result = []
  for candidate in candidates:
    start, end = candidate[0], candidate[1]
    sen = sentence[:]
    sen[start:end] = list(candidate[3])
    segs = word_tokenize(''.join(sen), engine='attacut')
    prod = 1
    for i in range(len(segs) - window_size):
      gram = segs[i:i+window_size]
      gram = ''.join(gram)
      ele = trigrame.dictt.get(gram)
      if ele != None:
        prod += ele
    result.append([candidate, prod])

  sent = sorted(result, key=lambda x: x[0][0])
  sen = sentence[:]

  i = 0
  while i < len(sent) - 1:
    a = [sent[i][0][0], sent[i][0][1]]
    b = [sent[i+1][0][0], sent[i+1][0][1]]
    if is_overlapped(a,b):
      if sent[i][1] >= sent[i+1][1]:
        sen[sent[i][0][0]:sent[i][0][1]] = list(sent[i][0][3])
      else:
        sen[sent[i+1][0][0]:sent[i+1][0][1]] = list(sent[i+1][0][3])
    else:
      sen[sent[i][0][0]:sent[i][0][1]] = list(sent[i][0][3])
      sen[sent[i+1][0][0]:sent[i+1][0][1]] = list(sent[i+1][0][3])
    i += 1
  return result, ''.join(sen)

#mock input
sentence = 'สงัดดีครับท่านสมาชิกชมรมคนชอบเกน วันนี่ผมจะมาเล่นเกใ'
candidates = [            
              [43,52,'มาเล่นเกใ','มาเล่นเกม'],  
              [33,41,'วันนี่ผม','วันนี้ผม'], #overlap 
              [0,14,'สงัดดีครับท่าน','สวัสดีครับท่าน'],
              [24,32,'คนชอบเกน','คนชอบเกม'], #overlap 
              [29,36,'เกน วัน','เกม วัน'], #overlap (choose higher prob (if equal, choose first occurence))
              [32,38,' วันนี่',' วันนี้'], #overlap 

]

result, sent = get_best_prop_sentence(sentence, candidates)
print(result)
print('\nold : ', sentence)
print('fixed : ', sent)


      

In [11]:
# # dict_new={"กม":{"เกม"}}
# from microdict import mdict

# dict_new = mdict.create("str:str", key_len = 200, val_len = 1000)
# dict_new["กม"] = "เกม"

In [15]:
import sqlite3
# del conn
conn = sqlite3.connect(':memory:')
conn.execute('''CREATE TABLE delt(K3Y TEXT NOT NULL,VAL TEXT NOT NULL);''')


In [22]:
k3y = 'อยู่ท้ง'
cursor = conn.execute(f'SELECT * from delt where K3Y = "{k3y}";')
for row in cursor:
  print(row)

('อยู่ท้ง', 'อยู่ท้อง')
('อยู่ท้ง', 'อยู่ทั้ง')
('อยู่ท้ง', 'อยู่ทิ้ง')


In [16]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = thai_letters
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    # transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    #replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    # inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes)#set(deletes + transposes + replaces + inserts)

In [17]:
def delet2dict(con,text):
  a = edits1(text)
  # print(len(a))
  for delet1 in a:
    try:
      con.execute(r"INSERT INTO delt (K3Y, VAL) VALUES ('"+delet1+"','"+text+"')")
    except:
      print(r"INSERT INTO delt (K3Y, VAL) VALUES ('"+delet1+"','"+text+"')")

  

In [18]:
uniword = unigrame.word
for text in tqdm(uniword[1:]):
  delet2dict(conn,text)

100%|██████████| 39963/39963 [00:02<00:00, 18584.06it/s]


In [19]:
uniword[0]

'ยโศธ'

In [20]:
len(uniword)

39964

In [21]:
biword = bigrame.word
for text in tqdm(biword):
  delet2dict(conn,text)

  8%|▊         | 107506/1326574 [00:09<01:53, 10713.08it/s]

INSERT INTO delt (K3Y, VAL) VALUES (']'<s/>',']='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES (']=<s/>',']='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES (']='<s>',']='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES (']='</>',']='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES (']='<s/',']='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES (']='s/>',']='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('='<s/>',']='<s/>')


 25%|██▍       | 328406/1326574 [00:29<01:30, 10982.22it/s]

INSERT INTO delt (K3Y, VAL) VALUES ('<s/_'','<s/>_'')
INSERT INTO delt (K3Y, VAL) VALUES ('s/>_'','<s/>_'')
INSERT INTO delt (K3Y, VAL) VALUES ('<s/>_','<s/>_'')
INSERT INTO delt (K3Y, VAL) VALUES ('<s>_'','<s/>_'')
INSERT INTO delt (K3Y, VAL) VALUES ('</>_'','<s/>_'')
INSERT INTO delt (K3Y, VAL) VALUES ('<s/>'','<s/>_'')


 25%|██▍       | 330685/1326574 [00:29<01:29, 11130.00it/s]

...

 79%|███████▊  | 1043774/1326574 [01:32<00:25, 11206.22it/s]

INSERT INTO delt (K3Y, VAL) VALUES ('s/>^^'','<s/>^^'')
INSERT INTO delt (K3Y, VAL) VALUES ('</>^^'','<s/>^^'')
INSERT INTO delt (K3Y, VAL) VALUES ('<s/^^'','<s/>^^'')
INSERT INTO delt (K3Y, VAL) VALUES ('<s/>^^','<s/>^^'')
INSERT INTO delt (K3Y, VAL) VALUES ('<s>^^'','<s/>^^'')
INSERT INTO delt (K3Y, VAL) VALUES ('<s/>^'','<s/>^^'')


 86%|████████▋ | 1144564/1326574 [01:41<00:16, 10866.14it/s]

INSERT INTO delt (K3Y, VAL) VALUES ('=_'<s/>','=_='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('=_='</>','=_='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('=_='<s/','=_='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('=_='<s>','=_='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('_='<s/>','=_='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('=_=<s/>','=_='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('=_='s/>','=_='<s/>')
INSERT INTO delt (K3Y, VAL) VALUES ('=='<s/>','=_='<s/>')


100%|██████████| 1326574/1326574 [01:58<00:00, 11217.07it/s]


In [149]:
import re
'ว่า".'.replace('"', '\\"')

'ว่า\\".'

In [154]:
"ว่า\"\."

'ว่า"\\.'

# to P time

In [ ]:
class candidater():
  def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = thai_letters
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    # transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    #replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    # inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes)#set(deletes + transposes + replaces + inserts)

  def delet2dict(dict_new,text):
    for delet1 in self.edits1(text):
      if dict_new.get(delet1) is None:
        dict_new.update({
            delet1:{text}
            })
      else:
        dict_new.get(delet1).add(text)

  def __init__(self,unigrame):
    self.unidict = unigrame.dictt
    uniword = unigrame.word
    dict_new = {}
    for text in tqdm(uniword):
      self.delet2dict(dict_new,text)
    self.dict_new = dict_new

  def candidate(self,word):
    set_candidate = set()

    if biword in dict_new.keys():
      set_candidate.update(self.dict_new[biword])
      
    del_set = edits1(biword)

    for word_del in del_set:
      
      if word_del in self.unidict.keys():
        set_candidate.add(word_del)

      if word_del in self.dict_new.keys():
        set_candidate.update(dict_new[word_del])
    return set_candidate

# plan

## First path

In [ ]:
from pythainlp import thai_digits, thai_letters
from pythainlp import word_tokenize
from pythainlp.spell import correct
import json

In [ ]:
list_word = word_tokenize("ถ้าเธอรักฉันจริงได้โปรดถิ้งฉันไปได้โปรดเดินหนี",engine="newmm")
# list_word = ["รับผิดขอบ"]
truesen = ""
for i in list_word:
    print(correct(i))
    truesen+= correct(i)
print(truesen)

ถ้า
เธอ
รัก
ฉัน
จริง
ได้
โปรด
ถิ้ง
ฉัน
ไป
ได้
โปรด
เดิน
หนี
ถ้าเธอรักฉันจริงได้โปรดถิ้งฉันไปได้โปรดเดินหนี


## Second path


```
input == dic # ไม่เจอแบบนี้
del(input) == dic #input หายไป 1 คำ 
input == del(dic) #input เขียนหายไป 1 คำ 
del(input) == del(dic) # input เขียนสลับที่ไป 1 ตัว

```



In [ ]:
def P(word,dict_word): 
  
  N=sum(dict_word.values())
  return dict_word[word] / N

def P_sort(word): 
  if word in unidict.keys():
    return P(word,unidict)
  elif word in bidict.keys():
    return P(word,bidict)

In [ ]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = thai_letters
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    # transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    #replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    # inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes)#set(deletes + transposes + replaces + inserts)

In [ ]:
unidict = unigrame.dictt
bidict = bigrame.dictt
#dict_new
def candidate(biword):
  set_candidate = set()

  if biword in dict_new.keys():
    set_candidate.update(dict_new[biword])
    
  del_set = edits1(biword)

  for word_del in del_set:
    
    if word_del in unidict.keys():
      set_candidate.add(word_del)

    if word_del in dict_new.keys():
      set_candidate.update(dict_new[word_del])

  return set_candidate

In [ ]:
unidict = unigrame.dictt
bidict = bigrame.dictt
#dict_new
def candidate(biword):
  set_candidate = set()

  if biword in dict_new.keys():
    set_candidate.update(dict_new[biword])
    
  del_set = edits1(biword)

  for word_del in del_set:

    if word_del in bidict.keys():
      set_candidate.add(word_del)
    if word_del in unidict.keys():
      set_candidate.add(word_del)

    if word_del in dict_new.keys():
      set_candidate.update(dict_new[word_del])

  return set_candidate

In [ ]:
def most_can(biword):
  set_can = candidate(biword)
  

In [ ]:
set_can = candidate("ฟาดพื้น")

In [ ]:
ddddddd = list(set_can)
ddddddd.sort(reverse = True,key=P_sort)

In [ ]:
ddddddd

['ฟาดพื้น']

In [ ]:
if ddddddd == []:
  return
  ddddddd[0]

'ฟาดพื้น'

In [ ]:
for candidate_word in set_can:
  print(candidate_word)
  if candidate_word in unidict:
    print(P(candidate_word,unidict))
  elif candidate_word in bidict:
    print(P(candidate_word,bidict))
  else:
    print("error")

ไก่เทอ
1.0856451921524138e-07
ไก่เอง
8.142338941143103e-08
ไก่ทอด
2.0355847352857756e-06
ไก่ของ
5.156814662723965e-07
ไก่สอง
5.428225960762069e-08
ไกรทอง
1.3570564901905172e-06
ไร่ทอง
5.428225960762069e-08


## Thrid path